## Import

In [57]:
import os
import re
import requests
import urllib
from bs4 import BeautifulSoup

### Get all pages

In [59]:
dir_of_images = './season_1/'
if not os.path.exists(dir_of_images):
    os.makedirs(dir_of_images)

pages = []
index = 'http://sny333.com/'
post_url = 'forum.php?mod=forumdisplay&fid=36&filter=typeid&typeid=365&mobile=2'
pre_page, next_page = post_url, post_url
while True:
    pages.append(index + next_page)
    res = requests.get(index + pre_page)
    soup = BeautifulSoup(res.text, 'html.parser')
    if len(soup.select('.nxt')) is 0:
        break
    next_page = soup.select('.nxt')[0]['href']
    pre_page = next_page

### Analysis urls

In [ ]:
def extract_imgs(url, page):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    pattern = 'attachment'
    x = 1
    for img_link in soup.select('img'):
        link = img_link.get('src')
        if re.search(pattern, link):
            image = requests.get(url + link)
            name = dir_of_images + page + '-' + str(x)
            if link.endswith('.jpg'):
                name += '.jpg'
            elif link.endswith('.gif'):
                name += '.gif'
            with open(name, 'wb') as f:
                f.write(image.content)
            x += 1


for i in range(len(pages)):
    res = requests.get(pages[i])
    soup = BeautifulSoup(res.text, 'html.parser')
    pattern = 'mod=viewthread'
    j = 0
    for link in soup.find_all('a'):
        link_text = link.get('href')
        if re.search(pattern, link_text):
            extract_imgs(index + link_text, str(i + 1) + '-' + str(j + 1))
            j += 1